Partie 1 : Preprocessing des données 
Phase de nettoyage des différents tableaux de données (PIB, PIB/hab, IDH, score, Dépenses) et mise sous le même format afin d'aboutir à "df_merged_final2", la table netoyée et fusionnant toutes les sources de données. 

In [ ]:
import sys
sys.path.append("../Scriptsfinal")  # dossier contenant tous tes scripts
import importlib
import medals
import pib_eu
import depenses
import Visu3
import Reg3
import merge_data
import Prepro3


importlib.reload(medals)
importlib.reload(pib_eu)
importlib.reload(depenses)
importlib.reload(Visu3)
importlib.reload(Reg3)
importlib.reload(merge_data)
importlib.reload(Prepro3)


# Importer les fonctions ou variables de chaque script
from medals import process_medals         # df_all_games, df_score
from pib_eu import process_pib_eu         # df_PIB_long
from pib_world import process_pib_world   # df_PIB_hab
from idh import process_idh               # df_IDH
from merge_data import merge_all           # df_merged_final2
from depenses import process_depenses     # df_merged_final avec dépenses
from Visu3 import plot_score_vs_spending
from Reg3 import run_ols





In [ ]:
df_all_games, df_score, df_athlete = process_medals()

df_score.head(10)

In [ ]:


df_PIB_long = process_pib_eu()
df_PIB_long.head(20)


In [ ]:
df_PIB_hab = process_pib_world()
df_PIB_hab.head(10)

In [ ]:
df_IDH = process_idh()
df_IDH.head(10)



In [ ]:
df_depenses = process_depenses()
df_depenses.head(10)

In [ ]:
df_merged_final2 = merge_all(df_score, df_PIB_hab, df_IDH, df_depenses)
df_merged_final2.head(20)


Phase 2 : Statistiques descriptives et réponse à notre problématique par des régressions et corrélations diverses

2.1 : Statistiques descriptives

In [ ]:
from Visu3 import plot_top_countries_medals
plot_top_countries_medals(df_all_games, top_n = 10) #top_n correspond au nombre de pays représentés en partant du meilleur


In [ ]:

from Visu3 import plot_hdi_over_time
countries = ["France", "United States", "India", "Switzerland", "Norway", "China", "Bresil"]
plot_hdi_over_time(df_IDH, countries)


2.2 : Analyse détaillée et premiers résultats de régression

Dans cette section, nous affinons encore notre dataframe en lui ajoutant deux variables (Taille de la délégation et score précédent), afin de procéder à différentes régressions. 

In [ ]:
import sys
sys.path.append("../Scriptsfinal")


import data_loading
import features
importlib.reload(data_loading)
importlib.reload(features)

from data_loading import clean_main_df, add_us_data
from features import add_previous_score, add_delegation_size
from Reg3 import correlation_analysis, run_ols, train_test_validation, add_lagged_depenses, run_2sls
from Visu3 import scatter_controls, plot_regression_results



In [ ]:

df = clean_main_df(df_merged_final2)
df = add_us_data(df)
df.head()


In [ ]:
df = add_previous_score(df, df_score)
df_Reg = add_delegation_size(df, df_athlete)
df_Reg.head(10)


In [ ]:

corr_simple, corr_hdi, corr_pib = correlation_analysis(df_Reg)
print(f"Corrélation simple entre Score et Dépenses (% PIB) : {corr_simple:.3f}")
print(f"Corrélation partielle Score-Dépenses contrôlant pour HDI : {corr_hdi:.3f}")
print(f"Corrélation partielle Score-Dépenses contrôlant pour PIB/hab : {corr_pib:.3f}")



In [ ]:
scatter_controls(df_Reg)


Analyse des régressions afin d'observer la corrélation entre le score et les dépenses (% du PIB) contrôlée par des variables sensées avoir un impact positif sur les performances aux JO

In [ ]:
df1 = df_Reg[df_Reg["Year"] == 2016]
model = run_ols(df1, ['Dépenses', 'HDI', 'score_précédent', 'taille_délégation'])
display(model.summary())

In [ ]:
print("=== OLS avec USA ===")
model_full = run_ols(df_Reg, ['Dépenses', 'HDI', 'score_précédent', 'taille_délégation'])
display(model_full.summary())

print("\n=== OLS sans USA ===")
df_no_usa = df_Reg[df_Reg['Country'] != 'United States']
model_no_usa = run_ols(df_no_usa, ['Dépenses', 'HDI', 'score_précédent', 'taille_délégation'])
display(model_no_usa.summary())



In [ ]:
res = train_test_validation(df_Reg[df_Reg['Country'] != 'United States'],
                            ['Dépenses', 'PIB_mean', 'score_précédent', 'taille_délégation'])

print("Validation train-test sur l'échantillon sans USA :")
print(f"R² = {res['R2']:.3f}, MSE = {res['MSE']:.3f}")
print(f"Coefficient de Dépenses estimé : {res['coef_depenses']:.3f}")



In [ ]:
plot_regression_results(
    y_full = df_Reg['Score'],
    fitted_full = model_full.fittedvalues,
    y_test = res['y_test'],
    y_pred = res['y_pred']
)


In [ ]:
# On ajoute les dépenses passées
df2sls = add_lagged_depenses(df_Reg.copy())

# Double MCO
first_stage, second_stage = run_2sls(df2sls)


print("\nPremière étape 2SLS :")
display(first_stage.summary())

print("\nDeuxième étape 2SLS :")
display(second_stage.summary())

# Comparaison avec OLS
print(f"\nComparaison : Coef Dépenses OLS = {model_full.params['Dépenses']:.3f}, 2SLS = {second_stage.params['Depenses_hat']:.3f}")


In [ ]:

#Partie 3 : Régressions en utilisant les dépenses sous forme de volume (En Milliards de SPA)

import sys
sys.path.append("../Scriptsfinal")


from Prepro3 import prepare_data_volume
from Visu3 import plot_score_vs_spending
from Reg3 import run_ols



In [ ]:
df = prepare_data_volume(
    df_merged_final2,
    df_PIB_long
)
df.head(20)


In [ ]:
plot_score_vs_spending(
    df,   
)


In [ ]:
import statsmodels.api as sm

X = df[['Dépenses_volume']]
X = sm.add_constant(X)
y = df['Score']

model = sm.OLS(y, X).fit()
print(model.summary())



In [ ]:
import importlib
import Prepro_long3
import pandas as pd
importlib.reload(Prepro_long3)
from Prepro_long3 import prepare_data_long
from Reg3 import run_ols


df_previous = df_Reg

df_long = prepare_data_long(df, df_previous)




In [ ]:
import statsmodels.api as sm

# Sélection des colonnes utilisées et suppression des NaN
df_long_simple = df_long[['Score', 'Dépenses_volume', 'score_précédent']].dropna()

# Préparer X et y
X = df_long_simple[['Dépenses_volume', 'score_précédent']]
X = sm.add_constant(X)  # ajoute l'intercept
y = df_long_simple['Score']

# Ajuster le modèle OLS
model_long = sm.OLS(y, X).fit()

# Afficher le résumé
print(model_long.summary())

